In [1]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor, AgentType, initialize_agent, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    JSONAgentOutputParser,
    ReActSingleInputOutputParser,
)
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun, tool
from langchain.tools.render import render_text_description_and_args, format_tool_to_openai_function
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.prompts import MessagesPlaceholder
from langchain.schema import ChatMessage, SystemMessage

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
)

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)

# OPENAI_GPT_MODEL = "gpt-4-1106-preview"
OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"

In [2]:
import re

mp_llm = ChatOpenAI(
    temperature=0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
)

llm = ChatOpenAI(
    temperature=0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY
)

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = [
    MPThermoExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=True)),
    MPElasticityExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=True)),
    MPSummaryExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=True)),
    # arxiv,
    # wikipedia,
]

prompt = hub.pull("hwchase17/react-multi-input-json")
prompt.messages[0].prompt.template = re.sub(
    r"\s+", " ",
    """You are a helpful data-aware agent that can consult materials-related
    data through Materials Project (MP) database, arXiv, and Wikipedia. Ask 
    user to clarify their queries if needed. Please note that you don't have 
    direct control to MP but through multiple assistant agents to help you. 
    You need to provide complete context in the input for them to do their job.
    """).strip().replace("\n", " ")[0] + prompt.messages[0].prompt.template

prompt = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm.bind(stop=["Observation"])
    # | map_reduce_chain  # TODO: Add map-reduce after LLM
    | JSONAgentOutputParser()
)


conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="chat_history")],
    # "early_stopping_method": 'generate',
    # "extra_prompt_messages": SystemMessage(
    #     content=re.sub(
    #         r"\s+", " ",
    #         """You are a helpful data-aware agent that can consult materials-related
    #         data through Materials Project (MP) database, arXiv, and Wikipedia. Ask 
    #         user to clarify their queries if needed. Please note that you don't have 
    #         direct control to MP but through multiple assistant agents to help you. 
    #         You need to provide complete context for them to do their job.
    #         """).strip().replace("\n", " ")[0]
    # )
}

agent_executor = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=5,
    memory=conversational_memory,
    # agent_kwargs=agent_kwargs,
    handle_parsing_errors=True,
)

# agent_executor = initialize_agent(
#     tools=tools,
#     llm=llm,
#     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     verbose=True,
#     max_iterations=5,
# )

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [3]:
agent_executor

AgentExecutor(memory=ConversationBufferWindowMemory(return_messages=True, memory_key='chat_history'), verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], template="Answer the following questions as best you can. You have access to the following tools:\n\nMPThermoExpert: Theromodynamics expert that has access to Materials Project thermo endpoint\nMPElasticityExpert: Elasticity expert that has access to Materials Project elasticity endpoint, including bulk, shear, and young's modulus, poisson ratio, and universal anisotropy index\nMPSummaryExpert: Summary expert that has access to Materials Project summary endpoint\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [MPThermoExpert, MPElasticityExpert, MPSummaryExpert]\nAction Input: the input to the action\nObs

In [4]:
agent_executor.invoke({
    "input": "Please give me the elastic tensor of NaCl"
})



> Entering new AgentExecutor chain...
To find the elastic tensor of NaCl, I need to access the Materials Project elasticity endpoint.

Action: MPElasticityExpert
Action Input: NaCl

> Entering new AgentExecutor chain...


/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Action:
```json
{
  "action": "search_materials_elasticity__get",
  "action_input": {
    "formula": "NaCl"
  }
}
```{"formula": "NaCl"}


Retrieving SummaryDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

Retrieving ElasticityDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

[{'formula_pretty': 'NaCl', 'material_id': 'mp-22851', 'elastic_tensor': {'raw': [[75.98221656025066, 0.7598099394417421, 0.7598099394417421, 4.510281037539697e-16, 4.440892098500625e-16, 4.440892098500625e-16], [0.7598099394417421, 75.98221656025066, 0.7598099394417424, 4.440892098500625e-16, 4.510281037539697e-16, 5.229936620407271e-16], [0.7598099394417421, 0.7598099394417424, 75.98221656025068, 4.440892098500626e-16, 4.440892098500626e-16, 4.510281037539696e-16], [4.510281037539697e-16, 4.440892098500625e-16, 4.440892098500626e-16, -2.6118973682358293, 0.0, -7.081558243585513e-32], [4.440892098500625e-16, 4.510281037539697e-16, 4.440892098500626e-16, 0.0, -2.6118973682358297, 7.249471490433169e-17], [4.440892098500625e-16, 5.229936620407271e-16, 4.510281037539696e-16, -7.081558243585513e-32, 7.249471490433169e-17, -2.611897368235813]], 'ieee_format': [[76.0, 1.0, 1.0, -0.0, 0.0, -0.0], [1.0, 76.0, 1.0, -0.0, -0.0, 0.0], [1.0, 1.0, 76.0, 0.0, -0.0, -0.0], [-0.0, -0.0, 0.0, -3.0, -0.

KeyboardInterrupt: 

In [ ]:
agent_executor.invoke({
    "input": "Could you tell me the 55 components of the elastic tensor of NaCl?"
})

In [ ]:
agent_executor.invoke({
    "input": "Sorry, I mean the 55 component in the elastic tensor of NaCl you retrieved"
})